<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- Creation of Ifrs Variables
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the creation of Ifrs Variable from the Scopes.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "4ImportScope-TechnicalMargin"

# Computing All Scopes

Since all the calculations defined in the various scopes are to be performed together, it is useful to introduce the scopes: <code>ComputeIfrsVarsActuals</code>, <code>ComputeIfrsVarsCashflows</code> and the <code>ComputeIfrsVarsOpenings</code> below, which given a certain Import Format and Import Storage, performs all the calculations (defined above) based on the corresponding data:

## Pv

In [0]:
public interface PvToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
  static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PvToIfrsVariable>(s => s.WithApplicability<EmptyPvIfrsVariable>(x => 
          !x.GetStorage().GetAllAocSteps(StructureTypes.AocPresentValue).Contains(x.Identity.AocStep)));

  IEnumerable<IfrsVariable> PvLocked => GetScope<PvLocked>(Identity).RepeatOnce().SelectMany(x =>
    x.PresentValues.Select(pv =>
    new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                      EstimateType = x.EstimateType, 
                      DataNode = x.Identity.DataNode, 
                      AocType = x.Identity.AocType, 
                      Novelty = x.Identity.Novelty, 
                      AccidentYear = pv.AccidentYear,
                      AmountType = pv.AmountType,
                      Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                      Partition = GetStorage().TargetPartition }));
  IEnumerable<IfrsVariable> PvCurrent => GetScope<PvCurrent>(Identity).RepeatOnce().SelectMany(x => 
    x.PresentValues.Select(pv =>
    new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                      EstimateType = x.EstimateType, 
                      DataNode = x.Identity.DataNode, 
                      AocType = x.Identity.AocType, 
                      Novelty = x.Identity.Novelty, 
                      AccidentYear = pv.AccidentYear,
                      AmountType = pv.AmountType,
                      Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                      Partition = GetStorage().TargetPartition }));
}

public interface EmptyPvIfrsVariable: PvToIfrsVariable{
  IEnumerable<IfrsVariable> PvToIfrsVariable.PvLocked => Enumerable.Empty<IfrsVariable>();
  IEnumerable<IfrsVariable> PvToIfrsVariable.PvCurrent => Enumerable.Empty<IfrsVariable>();
}


## Nominal

In [0]:
public interface NominalToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
      builder.ForScope<NominalToIfrsVariable>(s => s.WithApplicability<EmptyNominalToIfrsVariable>(x => 
          !x.GetStorage().GetAllAocSteps(StructureTypes.AocPresentValue).Contains(x.Identity.AocStep)));

    IEnumerable<IfrsVariable> CumulatedNominal => GetScope<CumulatedNominalBE>(Identity).RepeatOnce().SelectMany(x => 
        x.PresentValues.Select(pv => 
            new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                              EstimateType = x.EstimateType, 
                              DataNode = x.Identity.DataNode, 
                              AocType = x.Identity.AocType, 
                              Novelty = x.Identity.Novelty, 
                              AccidentYear = pv.AccidentYear,
                              AmountType = pv.AmountType,
                              Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                              Partition = GetStorage().TargetPartition}))
    .Concat(GetScope<CumulatedNominalRA>(Identity).RepeatOnce().SelectMany(x => 
        x.PresentValues.Select(pv => 
            new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                              EstimateType = x.EstimateType, 
                              DataNode = x.Identity.DataNode, 
                              AocType = x.Identity.AocType, 
                              Novelty = x.Identity.Novelty, 
                              AccidentYear = pv.AccidentYear,
                              AmountType = pv.AmountType,
                              Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                              Partition = GetStorage().TargetPartition})));
}

public interface EmptyNominalToIfrsVariable: NominalToIfrsVariable{
    IEnumerable<IfrsVariable> NominalToIfrsVariable.CumulatedNominal => Enumerable.Empty<IfrsVariable>();
}

## RiskAdjustment

In [0]:
public interface RaToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
  static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<RaToIfrsVariable>(s => s.WithApplicability<EmptyRaIfrsVariable>(x => 
            !x.GetStorage().GetAllAocSteps(StructureTypes.AocPresentValue).Contains(x.Identity.AocStep)));
    
    IEnumerable<IfrsVariable> RaCurrent => GetScope<RaCurrent>(Identity).RepeatOnce().SelectMany(x => 
        x.PresentValues.Select(pv => 
            new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                              EstimateType = x.EstimateType, 
                              DataNode = x.Identity.DataNode, 
                              AocType = x.Identity.AocType, 
                              Novelty = x.Identity.Novelty, 
                              AccidentYear = pv.AccidentYear,
                              AmountType = pv.AmountType,
                              Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                              Partition = GetStorage().TargetPartition}));
                        
    IEnumerable<IfrsVariable> RaLocked => GetScope<RaLocked>(Identity).RepeatOnce().SelectMany(x => 
        x.PresentValues.Select(pv => 
            new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                              EstimateType = x.EstimateType, 
                              DataNode = x.Identity.DataNode, 
                              AocType = x.Identity.AocType, 
                              Novelty = x.Identity.Novelty, 
                              AccidentYear = pv.AccidentYear,
                              AmountType = pv.AmountType,
                              Values = SetProjectionValue(pv.Value, x.Identity.ProjectionPeriod),
                              Partition = GetStorage().TargetPartition}));
}

public interface EmptyRaIfrsVariable: RaToIfrsVariable{
    IEnumerable<IfrsVariable> RaToIfrsVariable.RaCurrent => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> RaToIfrsVariable.RaLocked => Enumerable.Empty<IfrsVariable>();
}

## Actuals

In [0]:
public interface ActualToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> Actual => Identity.AocType == AocTypes.CF && Identity.Novelty == Novelties.C 
      ? GetScope<Actual>(Identity).Actuals.Select(written => 
      new IfrsVariable{ EstimateType = written.EstimateType,
                        DataNode = Identity.DataNode,
                        AocType = Identity.AocType,
                        Novelty = Identity.Novelty,
                        AccidentYear = written.AccidentYear,
                        AmountType = written.AmountType,
                        Values = SetProjectionValue(written.Value, Identity.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition })
      : Enumerable.Empty<IfrsVariable>();
                        
    IEnumerable<IfrsVariable> AdvanceActual => GetStorage().GetAllAocSteps(StructureTypes.AocAccrual).Contains(Identity.AocStep)
      ? GetScope<AdvanceActual>(Identity).Actuals.Select(advance => 
      new IfrsVariable{ EstimateType = advance.EstimateType,
                        DataNode = Identity.DataNode,
                        AocType = Identity.AocType,
                        Novelty = Identity.Novelty,
                        AccidentYear = advance.AccidentYear,
                        AmountType = advance.AmountType,
                        Values = SetProjectionValue(advance.Value, Identity.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition })
      : Enumerable.Empty<IfrsVariable>();

   IEnumerable<IfrsVariable> OverdueActual => GetStorage().GetAllAocSteps(StructureTypes.AocAccrual).Contains(Identity.AocStep)
      ? GetScope<OverdueActual>(Identity).Actuals.Select(overdue => 
      new IfrsVariable{ EstimateType = overdue.EstimateType,
                        DataNode = Identity.DataNode,
                        AocType = Identity.AocType,
                        Novelty = Identity.Novelty,
                        AccidentYear = overdue.AccidentYear,
                        AmountType = overdue.AmountType,
                        Values = SetProjectionValue(overdue.Value, Identity.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition })
      : Enumerable.Empty<IfrsVariable>();
}

## Deferrable

In [0]:
public interface DeferrableToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    protected string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod); 

    IEnumerable<IfrsVariable> Deferrable => GetScope<DiscountedDeferrable>(Identity).RepeatOnce()
                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = x.EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition });

    bool hasAmortizationStep => Deferrable.Where(iv => iv.Values != null).Any(iv => Math.Abs(iv.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision);

    IEnumerable<IfrsVariable> DeferrableAmFactor => Identity.AocType == AocTypes.AM && hasAmortizationStep
        ? GetScope<DiscountedAmortizationFactorForDeferrals>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = EstimateTypes.F,
                          EconomicBasis = EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          AmountType = x.EffectiveAmountType,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition })
        : Enumerable.Empty<IfrsVariable>();
}

## Premium Revenues

In [0]:
public interface RevenueToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<RevenueToIfrsVariable>(s => s
            .WithApplicability<EmptyRevenue>(x => !(x.Identity.ValuationApproach == ValuationApproaches.PAA && x.GetStorage().DataNodeDataBySystemName[x.Identity.DataNode].LiabilityType == LiabilityTypes.LRC)));

    protected string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod); 

    IEnumerable<IfrsVariable> Revenue => GetScope<PremiumRevenue>(Identity).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = x.EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition });
    
    private bool hasAmortizationStep => Revenue.Where(iv => iv.Values != null).Any(iv => Math.Abs(iv.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision);

    IEnumerable<IfrsVariable> RevenueAmFactor =>  Identity.AocType == AocTypes.AM && hasAmortizationStep
        ? GetScope<DiscountedAmortizationFactorForRevenues>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = EstimateTypes.F,
                          EconomicBasis = EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          AmountType = x.EffectiveAmountType,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition })
        : Enumerable.Empty<IfrsVariable>();
}
public interface EmptyRevenue : RevenueToIfrsVariable{
    IEnumerable<IfrsVariable> RevenueToIfrsVariable.Revenue => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> RevenueToIfrsVariable.RevenueAmFactor => Enumerable.Empty<IfrsVariable>();
}

## Experience Adj. for Premium

In [0]:
public interface EaForPremiumToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
  private string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
  IEnumerable<IfrsVariable> BeEAForPremium => Identity.AocType == AocTypes.CF && GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType != LiabilityTypes.LIC && !Identity.IsReinsurance
    ? GetScope<BeExperienceAdjustmentForPremium>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
      .Select(sc => new IfrsVariable{ EstimateType = sc.EstimateType, 
                                      DataNode = sc.Identity.DataNode, 
                                      AocType = sc.Identity.AocType, 
                                      Novelty = sc.Identity.Novelty, 
                                      EconomicBasis = sc.EconomicBasis,
                                      AmountType = sc.AmountType,
                                      Values = SetProjectionValue(sc.Value, sc.Identity.ProjectionPeriod),
                                      Partition = sc.GetStorage().TargetPartition })
    : Enumerable.Empty<IfrsVariable>();
    
  IEnumerable<IfrsVariable> ActEAForPremium => Identity.AocType == AocTypes.CF && GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType != LiabilityTypes.LIC && !Identity.IsReinsurance
    ? GetScope<ActualExperienceAdjustmentOnPremium>(Identity).RepeatOnce()
        .Select(sc => new IfrsVariable{ EstimateType = sc.EstimateType, 
                         DataNode = sc.Identity.DataNode, 
                         AocType = sc.Identity.AocType, 
                         Novelty = sc.Identity.Novelty, 
                         AmountType = sc.AmountType,
                         Values = SetProjectionValue(sc.Value, sc.Identity.ProjectionPeriod),
                         Partition = GetStorage().TargetPartition })
    : Enumerable.Empty<IfrsVariable>();
}

## Technical margin

In [0]:
public interface TmToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<TmToIfrsVariable>(s => s.WithApplicability<EmptyTmIfrsVariable>(x =>
            !x.GetStorage().GetAllAocSteps(StructureTypes.AocTechnicalMargin).Contains(x.Identity.AocStep)));

    private string economicBasis => Identity.ValuationApproach == ValuationApproaches.VFA ? EconomicBases.C : EconomicBases.L;
    private IEnumerable<string> amountTypesForTm => GetScope<TechnicalMarginAmountType>((Identity, EstimateTypes.C)).Values;
    // TODO: we need to think how to define the logic on when to compute LC for PAA-LRC
    // private bool hasTechnicalMargin => GetStorage().ImportFormat switch {
    //     ImportFormats.Cashflow => GetStorage().GetRawVariables(Identity.DataNode).Any(x => x.EstimateType == EstimateTypes.RA || 
    //         (x.EstimateType == EstimateTypes.BE && amountTypesForTm.Contains(x.AmountType))),
    //     _ => GetStorage().GetIfrsVariables(Identity.DataNode).Any(x => !GetStorage().EstimateTypesByImportFormat[ImportFormats.Actual].Contains(x.EstimateType) && 
    //         amountTypesForTm.Contains(x.AmountType))
    // };

    IEnumerable<IfrsVariable> Csms => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC || Identity.ValuationApproach == ValuationApproaches.PAA
        ? Enumerable.Empty<IfrsVariable>()
        : GetScope<ContractualServiceMargin>(Identity).RepeatOnce()
          .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
               DataNode = x.Identity.DataNode,
               AocType = x.Identity.AocType,
               Novelty = x.Identity.Novelty,
               EconomicBasis = economicBasis,
               Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
               Partition = GetStorage().TargetPartition
            });

     IEnumerable<IfrsVariable> Loss => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
        ? Enumerable.Empty<IfrsVariable>()
        : Identity.IsReinsurance 
           ? GetScope<LossRecoveryComponent>(Identity).RepeatOnce()
               .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                              DataNode = x.Identity.DataNode,
                                              AocType = x.Identity.AocType,
                                              Novelty = x.Identity.Novelty,
                                              EconomicBasis = economicBasis,
                                              Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                              Partition = GetStorage().TargetPartition
                                           })
           : GetScope<LossComponent>(Identity).RepeatOnce()
               .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                              DataNode = x.Identity.DataNode,
                                              AocType = x.Identity.AocType,
                                              Novelty = x.Identity.Novelty,
                                              EconomicBasis = economicBasis,
                                              Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                              Partition = GetStorage().TargetPartition
                                           });
    
    IEnumerable<IfrsVariable> AmortizationFactor =>  Identity.AocType == AocTypes.AM && Loss.Concat(Csms).Where(x => x.Values != null).Any(x => Math.Abs(x.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision)
        && GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LRC
        ? GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(economicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                           DataNode = x.Identity.Id.DataNode,
                                           AocType = x.Identity.Id.AocType,
                                           Novelty = x.Identity.Id.Novelty,
                                           AmountType = x.EffectiveAmountType,
                                           EconomicBasis = x.EconomicBasis,
                                           Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                                           Partition = GetStorage().TargetPartition
                                           })
        : Enumerable.Empty<IfrsVariable>();
}

public interface EmptyTmIfrsVariable: TmToIfrsVariable {
    IEnumerable<IfrsVariable> TmToIfrsVariable.Csms => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> TmToIfrsVariable.Loss => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> TmToIfrsVariable.AmortizationFactor => Enumerable.Empty<IfrsVariable>();
}